In [1]:
import requests
from bs4 import BeautifulSoup
import os

headers = {
    'authority': 'www.tranmererovers.co.uk',
    'pragma': 'no-cache',
    'cache-control': 'no-cache',
    'accept': '*/*',
    'x-requested-with': 'XMLHttpRequest',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 \
    (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
    'sec-fetch-site': 'same-origin',
}

In [2]:
ssn_urls = [
    {
        "season": "2023-24",
        "url": "https://www.tranmererovers.co.uk/teams/first-team/"
    },
    {
        "season": "2022-23",
        "url": "https://web.archive.org/web/20220804023137/https://www.tranmererovers.co.uk/teams/first-team/"
    },
    {
        "season": "2021-22",
        "url": "https://web.archive.org/web/20210804023137/https://www.tranmererovers.co.uk/teams/first-team/"
    },
    {
        "season": "2020-21",
        "url": "https://web.archive.org/web/20200929045703/https://www.tranmererovers.co.uk/teams/first-team/"
    },
    {
        "season": "2019-20",
        "url": "https://web.archive.org/web/20200509205704/https://www.tranmererovers.co.uk/teams/first-team/"
    },
    {
        "season": "2018-19",
        "url": "https://web.archive.org/web/20190401075011/https://www.tranmererovers.co.uk/teams/first-team/"
    },
    {
        "season": "2017-18",
        "url": "https://web.archive.org/web/20170712201358/https://tranmererovers.co.uk/team/player-profiles/"
    },
    {
        "season": "2002-03",
        "url": "http://web.archive.org/web/20020806214526/http://www.tranmererovers.co.uk/stats/view/player_profiles/0,,10365,00.html"
    }
]

In [21]:
player_images = []
def get_images(season, url):
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html.parser")

    if season in ["2018-19", "2019-20", "2020-21", "2021-22", "2022-23", "2023-24"]:
        players = soup.find_all("div", {"class": "player-container"})
    elif season == "2017-18":
        players = soup.find_all("div", {"data-anim-type": "bounceInLeft"})
    elif season == "2002-03":
        players = soup.find_all("td", {"class": "StatsSquadList", "align": "left"})

    dir = f"./player-images/{season}"
    if not os.path.exists(dir):
        os.makedirs(dir)

    for player in players:
        if season in ["2018-19", "2019-20", "2020-21", "2021-22", "2022-23", "2023-24"]:
            forename = player.find("span", "firstname").text
            surname = player.find("span", "surname").text
            player_name = f"{forename} {surname}"
        else:
            if season == "2017-18":
                player_name = player.find("h4").text
            elif season == "2002-03":
                try:
                    pl_url = "https://web.archive.org" + player.find("a", {"class": "StatsSquadList"})["href"]
                    r = requests.get(pl_url, headers = headers)
                    pl_soup = BeautifulSoup(r.text, "html.parser")
                    player_name = pl_soup.find("td", {"class": "CanvasBg"}).find_all("td", "StatsTitle")[1].text.strip()
                except:
                    next
            surname = player_name.split(" ")[-1]
            forename = " ".join(player_name.split(" ")[:-1])

        if season == "2023-24":
            img_url = "https://www.tranmererovers.co.uk" + player.find("img")["src"]
        elif season in ["2017-18", "2018-19", "2019-20", "2020-21", "2021-22", "2022-23"]:
            img_url = player.find("img")["src"]
        elif season == "2002-03":
            try:
                img_url = pl_soup.find("td", {"class": "CanvasBg"}).find_all("td", "StatPlayerTableBgCell")[1].find("img")["src"]
                if "TRFC-LARGE-logo.gif" in img_url:
                    next
            except:
                next

        if "https://web.archive.org" not in img_url:
            img_url = "https://web.archive.org" + img_url
        
        file_name = f"{season}-{surname}-{forename}.jpg".lower()
        file_path = f"{dir}/{file_name}"

        img_data = {
            "season": season.replace("-", "/"),
            "player_name": player_name,
            "img_url": img_url,
            "file_path": file_path
        }

        player_images.append(img_data)

    if season == "2002-03":
        for player in player_images:
            print(player["img_url"])
            r = requests.get(player["img_url"], allow_redirects = True)
            open(player["file_path"], 'wb').write(r.content)
            print(f"Added {player['player_name']}")

def get_all(seasons = None):
    for ssn in ssn_urls:
        if seasons and ssn["season"] not in seasons:
            print(f"Skipping {ssn['season']}")
            next
        else:
            print(f"Getting images for {ssn['season']}")
            get_images(ssn["season"], ssn["url"])

In [22]:
get_all(seasons = "2023-24")

Getting images for 2023-24
Skipping 2022-23
Skipping 2021-22
Skipping 2020-21
Skipping 2019-20
Skipping 2018-19
Skipping 2017-18
Skipping 2002-03


In [24]:
player_images

[{'season': '2023/24',
  'player_name': 'Luke McGee',
  'img_url': 'https://web.archive.orghttps://www.tranmererovers.co.uk/api/image/cropandgreyscale/2dd940cc-c6c3-4b01-9480-da211455ac3c/?preset=square&greyscale=false',
  'file_path': './player-images/2023-24/2023-24-mcgee-luke.jpg'},
 {'season': '2023/24',
  'player_name': 'Joe Murphy',
  'img_url': 'https://web.archive.orghttps://www.tranmererovers.co.uk/api/image/cropandgreyscale/b1be0fdb-adb0-45f5-8b47-7c14bdc7c2e8/?preset=square&greyscale=false',
  'file_path': './player-images/2023-24/2023-24-murphy-joe.jpg'},
 {'season': '2023/24',
  'player_name': 'Jake Leake',
  'img_url': 'https://web.archive.orghttps://www.tranmererovers.co.uk/static/images/players/male.png',
  'file_path': './player-images/2023-24/2023-24-leake-jake.jpg'},
 {'season': '2023/24',
  'player_name': 'Jordan Turnbull',
  'img_url': 'https://web.archive.orghttps://www.tranmererovers.co.uk/api/image/cropandgreyscale/42730c91-5a01-4186-8999-c04dad7b0add/?preset=sq

In [87]:
import pandas as pd

imgs = pd.DataFrame(player_images)
imgs.season = imgs.season.str.replace("-", "/")

ssns = pd.read_csv("https://raw.githubusercontent.com/petebrown/pre-2023-data-prep/main/data/results.csv", parse_dates = ["game_date"])[["game_date", "season"]].drop_duplicates()

apps = pd.read_csv("https://raw.githubusercontent.com/petebrown/pre-2023-data-prep/main/data/players_apps.csv", parse_dates = ["game_date"]).merge(ssns, on = "game_date", how = "left").query("season.isin(@imgs.season.unique())")[["season", "player_name"]].drop_duplicates()

apps.merge(imgs, on = ["season", "player_name"], how = "outer", indicator = True).query("_merge != 'both'").drop(columns = ["img_url", "file_path"])

,season,player_name,_merge
4,2002/03,Micky Mellon,left_only
29,2002/03,Keith Welch,left_only
46,2002/03,Eric Nixon,left_only
47,2002/03,Pat McGibbon,left_only
48,2002/03,Ian Feuer,left_only
61,2002/03,Gary Jones,left_only
62,2002/03,Tom Curtis,left_only
71,2002/03,Christian Edwards,left_only
76,2002/03,Phil Whitehead,left_only
81,2002/03,Sean Connelly,left_only
